In [1]:
# import dependencies 
# from selenium import webdriver
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import re
from urllib.request import urlopen
# import time
import pandas as pd
import numpy as np

# set options
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)

In [2]:
executable_path = {'executable_path': 'C:\\Users\\Trisarah\\Desktop\chromedriver.exe'} 
browser= Browser('chrome', **executable_path, headless=False)
base_url = 'https://www.ulta.com'

In [3]:
# # set urls
ulta_urls = ['https://www.ulta.com/skin-care-cleansers?N=2794',
              'https://www.ulta.com/skin-care-moisturizers?N=2796',
              'https://www.ulta.com/skin-care-treatment-serums?N=27cs',
              'https://www.ulta.com/skin-care-eye-treatments?N=270k',
              'https://www.ulta.com/skin-care-suncare?N=27fe',
              'https://www.ulta.com/makeup-face?N=26y3',
              'https://www.ulta.com/makeup-eyes?N=26yd',
              'https://www.ulta.com/makeup-lips?N=26yq',
              'https://www.ulta.com/hair-shampoo-conditioner?N=27ih',
              'https://www.ulta.com/hair-treatment?N=26xy',
              'https://www.ulta.com/hair-styling-products?N=26xf',
              'https://www.ulta.com/hair-color?N=26xs',
              'https://www.ulta.com/bath-body-bath-shower?N=26uy',
              'https://www.ulta.com/bath-body-body-moisturizers?N=26v3',
              'https://www.ulta.com/bath-body-hand-foot-care?N=27ic',
              'https://www.ulta.com/bath-body-personal-care?N=27i3',
              'https://www.ulta.com/bath-body-suncare?N=276b',
              'https://www.ulta.com/skin-care-cleansers?N=2794&No=96&Nrpp=96',
              'https://www.ulta.com/skin-care-moisturizers?N=2796&No=96&Nrpp=96',
              'https://www.ulta.com/skin-care-treatment-serums?N=27cs&No=96&Nrpp=96',
              'https://www.ulta.com/skin-care-eye-treatments?N=270k&No=96&Nrpp=96',
              'https://www.ulta.com/skin-care-suncare?N=27fe&No=96&Nrpp=96',
              'https://www.ulta.com/makeup-face?N=26y3&No=96&Nrpp=96',
              'https://www.ulta.com/makeup-eyes?N=26yd&No=96&Nrpp=96',
              'https://www.ulta.com/makeup-lips?N=26yq&No=96&Nrpp=96',
              'https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=96&Nrpp=96',
              'https://www.ulta.com/hair-treatment?N=26xy&No=96&Nrpp=96',
              'https://www.ulta.com/hair-styling-products?N=26xf&No=96&Nrpp=96',
              'https://www.ulta.com/hair-color?N=26xs&No=96&Nrpp=96',
              'https://www.ulta.com/bath-body-bath-shower?N=26uy&No=96&Nrpp=96',
              'https://www.ulta.com/bath-body-body-moisturizers?N=26v3&No=96&Nrpp=96',
              'https://www.ulta.com/bath-body-hand-foot-care?N=27ic&No=96&Nrpp=96',
              'https://www.ulta.com/bath-body-personal-care?N=27i3&No=96&Nrpp=96',
              'https://www.ulta.com/bath-body-suncare?N=276b&No=96&Nrpp=96']

In [4]:
# create empty df
# products_df = pd.DataFrame(columns=['url', 'id', 'description', 'brand'])
products_df = pd.DataFrame()
rating_df = pd.DataFrame()
reviews_df = pd.DataFrame()
price_df = pd.DataFrame()

In [5]:
# loop through the ulta urls
for url in ulta_urls:
    review_number = []
    browser.visit(url)
    r = requests.get(url, timeout=5)
    
    # parse html source
    html = browser.html
    data = bs(html, 'html.parser')
    
    # get product title, review info, and ratings info 
    prod_title = data.find_all('h4', attrs={'prod-title'})
    reviews = data.find_all('span', attrs={'prodCellReview'})
    ratings_pricing_info = data.find_all('div', attrs={'productQvContainer'})
    
    # get the number of ratings 
    for reviewnum in reviews:
        review_hrefs = reviewnum.findAll('a')
        for final_review_href in review_hrefs:
            review_href = final_review_href['href']
            review_num_text = reviewnum.getText()
            number_of_reviews = re.findall(r"\(([^\)]+)\)", review_num_text)
            reviews_df = reviews_df.append({"number of ratings": number_of_reviews, "href": review_href}, ignore_index=True)
    
    for rating in ratings_pricing_info:
        ratings_div = rating.find_all('label', class_='sr-only')[0]
        rating_df = rating_df.append({"rating": ratings_div}, ignore_index=True)
        
#     loop through the links and generate unique urls for each product
    for link in prod_title:
        find_href = link.findAll('a')
        
        # find product url, description, brand
        for a in find_href:
            href = a['href']
            new_link = base_url + a['href']
            description = re.findall(r"\/([\s\S]*?)\?", href)[0]
            brand = a.getText().split('\n\t\t\t\t')[1]
            id_number = new_link.rsplit('=', 1)[+1]
            
            #needs to be in find_href for loop
        products_df = products_df.append({"url": new_link, "description": description, "brand": brand, 
                                  "id": id_number, "href": href}, ignore_index=True)

In [6]:
reviews_df.head()

,href,number of ratings
0,/purity-made-simple-one-step-facial-cleanser?p...,[3881 reviews]
1,/special-cleansing-gel?productId=VP12697,[2202 reviews]
2,/take-day-off-cleansing-balm?productId=xlsImpp...,[1793 reviews]
3,/liquid-facial-soap-mild?productId=xlsImpprod1...,[601 reviews]
4,/microdelivery-exfoliating-facial-wash?product...,[672 reviews]


In [7]:
merged_product_pd = pd.merge(products_df, reviews_df, how='outer', on='href')
merged_product_pd.head()

,brand,description,href,id,url,number of ratings
0,Philosophy,purity-made-simple-one-step-facial-cleanser,/purity-made-simple-one-step-facial-cleanser?p...,xlsImpprod1490012,https://www.ulta.com/purity-made-simple-one-st...,[3881 reviews]
1,Dermalogica,special-cleansing-gel,/special-cleansing-gel?productId=VP12697,VP12697,https://www.ulta.com/special-cleansing-gel?pro...,[2202 reviews]
2,Clinique,take-day-off-cleansing-balm,/take-day-off-cleansing-balm?productId=xlsImpp...,xlsImpprod11771009,https://www.ulta.com/take-day-off-cleansing-ba...,[1793 reviews]
3,Clinique,take-day-off-cleansing-balm,/take-day-off-cleansing-balm?productId=xlsImpp...,xlsImpprod11771009,https://www.ulta.com/take-day-off-cleansing-ba...,[1793 reviews]
4,Clinique,take-day-off-cleansing-balm,/take-day-off-cleansing-balm?productId=xlsImpp...,xlsImpprod11771009,https://www.ulta.com/take-day-off-cleansing-ba...,[1793 reviews]


In [8]:
# merge ratings with merged product pd
merged_product_pd = merged_product_pd.merge(rating_df, left_index=True, right_index=True)

In [9]:
# create empty ingredients df
ingredients_df = pd.DataFrame()
price_df = pd.DataFrame()

In [10]:
for i in products_df['url']:
    try:
        r = requests.get(url, timeout=5)
        html = urlopen(i).read()
        soup = bs(html, 'html.parser')
        
        # get product info, ingredients and coupon info on ulta url
        product_info = soup.findAll('div', attrs={'class': 'ProductDetail__ingredients'})
        price_info = soup.findAll('div', attrs={'class': 'ProductPricingPanel'})

#         # loop through product info to get ingredients 
        for ingredients in product_info:
            # will print out div with ingredients
            content = ingredients.find_all('div', {'class' : 'ProductDetail__productContent'})
            
            for ing_list in content:
                ing_text = ing_list.getText()
                x = ing_text.split(', ')
                ingredients_df = ingredients_df.append({"ingredients":x, "url": i}, ignore_index=True)
                
        for price in price_info:
            price_text = price.getText()
            price_df = price_df.append({"price":price_text, "url": i}, ignore_index=True)
        
    except:
        np.nan
browser.quit()

In [11]:
# merge price with the df 
merged_product_pd = pd.merge(merged_product_pd, price_df, how='outer', on='url')

In [12]:
# merge ingredietns with df
final_product_df = pd.merge(merged_product_pd, ingredients_df, how='outer', on='url')

In [43]:
# pd.set_option('display.max_rows', None)
# convert the rating to a string
final_product_df['rating'] = final_product_df['rating'].astype(str)

# extract the float rating 
final_product_df['rating'] = final_product_df['rating'].str.extract('(\d.\d\d)', expand=True)

In [14]:
final_product_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7739 entries, 0 to 7738
Data columns (total 9 columns):
brand                7130 non-null object
description          7130 non-null object
href                 7130 non-null object
id                   7130 non-null object
url                  7739 non-null object
number of ratings    6844 non-null object
rating               6895 non-null object
price                7738 non-null object
ingredients          7396 non-null object
dtypes: object(9)
memory usage: 604.6+ KB


In [15]:
# drop duplicate rows based on the href column in df
final_product_df = final_product_df.drop_duplicates(subset="href")

In [16]:
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_colwidth', 5000)
# pd.set_option('display.max_rows', None)

In [17]:
unique_list = []
for x in final_product_df['ingredients'].dropna():
    for k in x:
        if k not in unique_list:
            unique_list.append(k) 

In [18]:
final_product_df['number_of_ingredients'] = final_product_df['ingredients'].str.len()

In [44]:
final_product_df

,brand,description,href,id,url,number of ratings,rating,price,ingredients,number_of_ingredients,test
0,Philosophy,purity-made-simple-one-step-facial-cleanser,/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,xlsImpprod1490012,https://www.ulta.com/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,3881 reviews,4.60,Price$36.00,"[Water, Sodium Lauroamphoacetate, Sodium Trideceth Sulfate, Limnanthes Alba (Meadowfoam) Seed Oil, Coco-Glucoside, Coconut Alcohol, PEG-120 Methyl Glucose Dioleate, Glycerin, Carbomer, Isopropyl Alcohol, Polysorbate 20, Citric Acid, Imidazolidinyl Urea, Methylparaben, Aniba Rosaeodora (Rosewood) Wood Oil, Propylparaben, Triethanolamine, Amyris Balsamifera Bark Oil, Bulnesia Sarmientoi Wood Oil, Cymbopogon Martini Oil, Pelargonium Graveolens Flower Oil, Rosa Centifolia Flower Oil, Santalum Album (Sandalwood) Oil, Daucus Carota Sativa (Carrot) Seed Oil, Mimosa Tenuiflora Bark Extract, Ormenis Multicaulis Oil, Piper Nigrum (Pepper) Fruit Oil, Salvia Sclarea (Clary) Oil, Fd&C Yellow No. 5 (Ci 19140).]",29.0,NaN
1,Dermalogica,special-cleansing-gel,/special-cleansing-gel?productId=VP12697,VP12697,https://www.ulta.com/special-cleansing-gel?productId=VP12697,2202 reviews,4.60,Price$39.00,"[Water/Aqua/Eau, Sodium Laureth Sulfate, Disodium Lauroamphodiacetate, Sodium Trideceth Sulfate, Hexylene Glycol, Hydroxyethylcellulose, Lavandula Angustifolia (Lavender) Extract, Melissa Officinalis Leaf Extract, Quillaja Saponaria Bark Extract, Lavandula Angustifolia (Lavender) Oil, PPG-5-Ceteth-20, Citric Acid, Sodium Chloride, Phenoxyethanol, Methylparaben, Propylparaben.]",16.0,NaN
2,Clinique,take-day-off-cleansing-balm,/take-day-off-cleansing-balm?productId=xlsImpprod11771009,xlsImpprod11771009,https://www.ulta.com/take-day-off-cleansing-balm?productId=xlsImpprod11771009,1793 reviews,4.70,Price$30.00,"[Ethylhexyl Palmitate, Carthamus Tinctorius (Safflower) Seed Oil, Caprylic / Capric Triglyceride, Sorbeth-30 Tetraoleate, Polyethylene, PEG-5 Glyceryl Triisostearate, Water / Aqua / Eau, Tocopherol, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",9.0,NaN
18,Clinique,liquid-facial-soap-mild,/liquid-facial-soap-mild?productId=xlsImpprod10791729,xlsImpprod10791729,https://www.ulta.com/liquid-facial-soap-mild?productId=xlsImpprod10791729,601 reviews,4.50,Price$18.50,"[Water / Aqua / Eau, Sodium Laureth Sulfate, Sodium Chloride, Cocamidopropyl Hydroxysultaine, Lauramidopropyl Betaine, Sodium Cocoyl Sarcosinate, Tea-Cocoyl Glutamate, Di-PPG-2 Myreth-10 Adipate, Aloe Barbadensis Leaf Juice, PEG-120 Methyl Glucose Dioleate, Sucrose, Sodium Hyaluronate, Cetyl Triethylmonium Dimethicone PEG-8 Succinate, Butylene Glycol, Hexylene Glycol, Polyquaternium-7, Laureth-2, Caprylyl Glycol, Sodium Sulfate, Tocopheryl Acetate, EDTA, Disodium EDTA, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",23.0,NaN
19,Philosophy,microdelivery-exfoliating-facial-wash,/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,xlsImpprod15641100,https://www.ulta.com/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,672 reviews,4.00,Price$28.00,"[Aqua/Water/Eau, Solum Diatomeae/Diatomaceous Earth/Terre De Diatomees, Sodium Cocoyl Apple Amino Acids, Glycerin,Acrylates/Steareth-20 Methacrylate Copolymer, Acrylates Copolymer, Decyl Glucoside, Disodium Cocoamphodiacetate, Phenoxyethanol, Lauryl Glucoside, Lithium Magnesium Sodium Silicate, Polysorbate 20, Hexylene Glycol, Sodium Hydroxide, Citrus Aurantium Dulcis (Orange) Peel Extract, Limonene, Decylene Glycol, Ethylhexylglycerin, Panthenol, Tetrasodium Edta, 1,2-Hexanediol, Ascorbyl Glucoside, Borago Officinalis Seed Oil, Tocop

In [20]:
# updated first index of number of ratings to a string
final_product_df['number of ratings'] = final_product_df['number of ratings'].str[0]

In [22]:
# drop old number of reviews column
# final_product_df.drop(columns='number of rating', inplace=True)

In [23]:
# create new df witih dropped duplicate rows
final_product_df2 = final_product_df.dropna()

In [48]:
final_product_df2

,brand,description,href,id,url,number of ratings,rating,price,ingredients,number_of_ingredients,number_of_reviews,price_USD
0,Philosophy,purity-made-simple-one-step-facial-cleanser,/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,xlsImpprod1490012,https://www.ulta.com/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,3881 reviews,4.60,Price$36.00,"[Water, Sodium Lauroamphoacetate, Sodium Trideceth Sulfate, Limnanthes Alba (Meadowfoam) Seed Oil, Coco-Glucoside, Coconut Alcohol, PEG-120 Methyl Glucose Dioleate, Glycerin, Carbomer, Isopropyl Alcohol, Polysorbate 20, Citric Acid, Imidazolidinyl Urea, Methylparaben, Aniba Rosaeodora (Rosewood) Wood Oil, Propylparaben, Triethanolamine, Amyris Balsamifera Bark Oil, Bulnesia Sarmientoi Wood Oil, Cymbopogon Martini Oil, Pelargonium Graveolens Flower Oil, Rosa Centifolia Flower Oil, Santalum Album (Sandalwood) Oil, Daucus Carota Sativa (Carrot) Seed Oil, Mimosa Tenuiflora Bark Extract, Ormenis Multicaulis Oil, Piper Nigrum (Pepper) Fruit Oil, Salvia Sclarea (Clary) Oil, Fd&C Yellow No. 5 (Ci 19140).]",29.0,3881,36.00
1,Dermalogica,special-cleansing-gel,/special-cleansing-gel?productId=VP12697,VP12697,https://www.ulta.com/special-cleansing-gel?productId=VP12697,2202 reviews,4.60,Price$39.00,"[Water/Aqua/Eau, Sodium Laureth Sulfate, Disodium Lauroamphodiacetate, Sodium Trideceth Sulfate, Hexylene Glycol, Hydroxyethylcellulose, Lavandula Angustifolia (Lavender) Extract, Melissa Officinalis Leaf Extract, Quillaja Saponaria Bark Extract, Lavandula Angustifolia (Lavender) Oil, PPG-5-Ceteth-20, Citric Acid, Sodium Chloride, Phenoxyethanol, Methylparaben, Propylparaben.]",16.0,2202,39.00
2,Clinique,take-day-off-cleansing-balm,/take-day-off-cleansing-balm?productId=xlsImpprod11771009,xlsImpprod11771009,https://www.ulta.com/take-day-off-cleansing-balm?productId=xlsImpprod11771009,1793 reviews,4.70,Price$30.00,"[Ethylhexyl Palmitate, Carthamus Tinctorius (Safflower) Seed Oil, Caprylic / Capric Triglyceride, Sorbeth-30 Tetraoleate, Polyethylene, PEG-5 Glyceryl Triisostearate, Water / Aqua / Eau, Tocopherol, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",9.0,1793,30.00
18,Clinique,liquid-facial-soap-mild,/liquid-facial-soap-mild?productId=xlsImpprod10791729,xlsImpprod10791729,https://www.ulta.com/liquid-facial-soap-mild?productId=xlsImpprod10791729,601 reviews,4.50,Price$18.50,"[Water / Aqua / Eau, Sodium Laureth Sulfate, Sodium Chloride, Cocamidopropyl Hydroxysultaine, Lauramidopropyl Betaine, Sodium Cocoyl Sarcosinate, Tea-Cocoyl Glutamate, Di-PPG-2 Myreth-10 Adipate, Aloe Barbadensis Leaf Juice, PEG-120 Methyl Glucose Dioleate, Sucrose, Sodium Hyaluronate, Cetyl Triethylmonium Dimethicone PEG-8 Succinate, Butylene Glycol, Hexylene Glycol, Polyquaternium-7, Laureth-2, Caprylyl Glycol, Sodium Sulfate, Tocopheryl Acetate, EDTA, Disodium EDTA, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",23.0,601,18.50
19,Philosophy,microdelivery-exfoliating-facial-wash,/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,xlsImpprod15641100,https://www.ulta.com/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,672 reviews,4.00,Price$28.00,"[Aqua/Water/Eau, Solum Diatomeae/Diatomaceous Earth/Terre De Diatomees, Sodium Cocoyl Apple Amino Acids, Glycerin,Acrylates/Steareth-20 Methacrylate Copolymer, Acrylates Copolymer, Decyl Glucoside, Disodium Cocoamphodiacetate, Phenoxyethanol, Lauryl Glucoside, Lithium Magnesium Sodium Silicate, Polysorbate 20, Hexylene Glycol, Sodium Hydroxide, Citrus Aurantium Dulcis (Orange) Peel Extract, Limonene, Decylene Glycol, Ethylhexylglycerin, Panthenol, Tetrasodium Edta, 1,2-Hexanediol, Asco

In [50]:
final_product_df2.to_csv('ulta_makeup.csv')

In [39]:
final_product_df2['number of ratings'] = final_product_df2['number of ratings'].astype(str)
final_product_df2['number_of_reviews'] = final_product_df2['number of ratings'].str.extract('(\d+)')

C:\Users\Trisarah\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Trisarah\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [40]:
final_product_df2['price'] = final_product_df2['price'].astype(str)
final_product_df2['price_USD'] = final_product_df2['price'].str.extract('(\d+.\d+)')

C:\Users\Trisarah\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Trisarah\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [41]:
final_product_df.to_csv('ulta_makeup.csv')

In [51]:
final_product_df2

,brand,description,href,id,url,number of ratings,rating,price,ingredients,number_of_ingredients,number_of_reviews,price_USD
0,Philosophy,purity-made-simple-one-step-facial-cleanser,/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,xlsImpprod1490012,https://www.ulta.com/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,3881 reviews,4.60,Price$36.00,"[Water, Sodium Lauroamphoacetate, Sodium Trideceth Sulfate, Limnanthes Alba (Meadowfoam) Seed Oil, Coco-Glucoside, Coconut Alcohol, PEG-120 Methyl Glucose Dioleate, Glycerin, Carbomer, Isopropyl Alcohol, Polysorbate 20, Citric Acid, Imidazolidinyl Urea, Methylparaben, Aniba Rosaeodora (Rosewood) Wood Oil, Propylparaben, Triethanolamine, Amyris Balsamifera Bark Oil, Bulnesia Sarmientoi Wood Oil, Cymbopogon Martini Oil, Pelargonium Graveolens Flower Oil, Rosa Centifolia Flower Oil, Santalum Album (Sandalwood) Oil, Daucus Carota Sativa (Carrot) Seed Oil, Mimosa Tenuiflora Bark Extract, Ormenis Multicaulis Oil, Piper Nigrum (Pepper) Fruit Oil, Salvia Sclarea (Clary) Oil, Fd&C Yellow No. 5 (Ci 19140).]",29.0,3881,36.00
1,Dermalogica,special-cleansing-gel,/special-cleansing-gel?productId=VP12697,VP12697,https://www.ulta.com/special-cleansing-gel?productId=VP12697,2202 reviews,4.60,Price$39.00,"[Water/Aqua/Eau, Sodium Laureth Sulfate, Disodium Lauroamphodiacetate, Sodium Trideceth Sulfate, Hexylene Glycol, Hydroxyethylcellulose, Lavandula Angustifolia (Lavender) Extract, Melissa Officinalis Leaf Extract, Quillaja Saponaria Bark Extract, Lavandula Angustifolia (Lavender) Oil, PPG-5-Ceteth-20, Citric Acid, Sodium Chloride, Phenoxyethanol, Methylparaben, Propylparaben.]",16.0,2202,39.00
2,Clinique,take-day-off-cleansing-balm,/take-day-off-cleansing-balm?productId=xlsImpprod11771009,xlsImpprod11771009,https://www.ulta.com/take-day-off-cleansing-balm?productId=xlsImpprod11771009,1793 reviews,4.70,Price$30.00,"[Ethylhexyl Palmitate, Carthamus Tinctorius (Safflower) Seed Oil, Caprylic / Capric Triglyceride, Sorbeth-30 Tetraoleate, Polyethylene, PEG-5 Glyceryl Triisostearate, Water / Aqua / Eau, Tocopherol, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",9.0,1793,30.00
18,Clinique,liquid-facial-soap-mild,/liquid-facial-soap-mild?productId=xlsImpprod10791729,xlsImpprod10791729,https://www.ulta.com/liquid-facial-soap-mild?productId=xlsImpprod10791729,601 reviews,4.50,Price$18.50,"[Water / Aqua / Eau, Sodium Laureth Sulfate, Sodium Chloride, Cocamidopropyl Hydroxysultaine, Lauramidopropyl Betaine, Sodium Cocoyl Sarcosinate, Tea-Cocoyl Glutamate, Di-PPG-2 Myreth-10 Adipate, Aloe Barbadensis Leaf Juice, PEG-120 Methyl Glucose Dioleate, Sucrose, Sodium Hyaluronate, Cetyl Triethylmonium Dimethicone PEG-8 Succinate, Butylene Glycol, Hexylene Glycol, Polyquaternium-7, Laureth-2, Caprylyl Glycol, Sodium Sulfate, Tocopheryl Acetate, EDTA, Disodium EDTA, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",23.0,601,18.50
19,Philosophy,microdelivery-exfoliating-facial-wash,/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,xlsImpprod15641100,https://www.ulta.com/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,672 reviews,4.00,Price$28.00,"[Aqua/Water/Eau, Solum Diatomeae/Diatomaceous Earth/Terre De Diatomees, Sodium Cocoyl Apple Amino Acids, Glycerin,Acrylates/Steareth-20 Methacrylate Copolymer, Acrylates Copolymer, Decyl Glucoside, Disodium Cocoamphodiacetate, Phenoxyethanol, Lauryl Glucoside, Lithium Magnesium Sodium Silicate, Polysorbate 20, Hexylene Glycol, Sodium Hydroxide, Citrus Aurantium Dulcis (Orange) Peel Extract, Limonene, Decylene Glycol, Ethylhexylglycerin, Panthenol, Tetrasodium Edta, 1,2-Hexanediol, Asco

In [52]:
# ingredient_list = pd.DataFrame(final_product_df['ingredients'].tolist())

In [53]:
# final_product_df['ingredients']

In [32]:
# final_product_df['test'] = final_product_df['ingredients'].str.split(',', expand=True)

In [54]:
# final_product_df.drop(columns='test')

In [55]:
final_product_df2

,brand,description,href,id,url,number of ratings,rating,price,ingredients,number_of_ingredients,number_of_reviews,price_USD
0,Philosophy,purity-made-simple-one-step-facial-cleanser,/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,xlsImpprod1490012,https://www.ulta.com/purity-made-simple-one-step-facial-cleanser?productId=xlsImpprod1490012,3881 reviews,4.60,Price$36.00,"[Water, Sodium Lauroamphoacetate, Sodium Trideceth Sulfate, Limnanthes Alba (Meadowfoam) Seed Oil, Coco-Glucoside, Coconut Alcohol, PEG-120 Methyl Glucose Dioleate, Glycerin, Carbomer, Isopropyl Alcohol, Polysorbate 20, Citric Acid, Imidazolidinyl Urea, Methylparaben, Aniba Rosaeodora (Rosewood) Wood Oil, Propylparaben, Triethanolamine, Amyris Balsamifera Bark Oil, Bulnesia Sarmientoi Wood Oil, Cymbopogon Martini Oil, Pelargonium Graveolens Flower Oil, Rosa Centifolia Flower Oil, Santalum Album (Sandalwood) Oil, Daucus Carota Sativa (Carrot) Seed Oil, Mimosa Tenuiflora Bark Extract, Ormenis Multicaulis Oil, Piper Nigrum (Pepper) Fruit Oil, Salvia Sclarea (Clary) Oil, Fd&C Yellow No. 5 (Ci 19140).]",29.0,3881,36.00
1,Dermalogica,special-cleansing-gel,/special-cleansing-gel?productId=VP12697,VP12697,https://www.ulta.com/special-cleansing-gel?productId=VP12697,2202 reviews,4.60,Price$39.00,"[Water/Aqua/Eau, Sodium Laureth Sulfate, Disodium Lauroamphodiacetate, Sodium Trideceth Sulfate, Hexylene Glycol, Hydroxyethylcellulose, Lavandula Angustifolia (Lavender) Extract, Melissa Officinalis Leaf Extract, Quillaja Saponaria Bark Extract, Lavandula Angustifolia (Lavender) Oil, PPG-5-Ceteth-20, Citric Acid, Sodium Chloride, Phenoxyethanol, Methylparaben, Propylparaben.]",16.0,2202,39.00
2,Clinique,take-day-off-cleansing-balm,/take-day-off-cleansing-balm?productId=xlsImpprod11771009,xlsImpprod11771009,https://www.ulta.com/take-day-off-cleansing-balm?productId=xlsImpprod11771009,1793 reviews,4.70,Price$30.00,"[Ethylhexyl Palmitate, Carthamus Tinctorius (Safflower) Seed Oil, Caprylic / Capric Triglyceride, Sorbeth-30 Tetraoleate, Polyethylene, PEG-5 Glyceryl Triisostearate, Water / Aqua / Eau, Tocopherol, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",9.0,1793,30.00
18,Clinique,liquid-facial-soap-mild,/liquid-facial-soap-mild?productId=xlsImpprod10791729,xlsImpprod10791729,https://www.ulta.com/liquid-facial-soap-mild?productId=xlsImpprod10791729,601 reviews,4.50,Price$18.50,"[Water / Aqua / Eau, Sodium Laureth Sulfate, Sodium Chloride, Cocamidopropyl Hydroxysultaine, Lauramidopropyl Betaine, Sodium Cocoyl Sarcosinate, Tea-Cocoyl Glutamate, Di-PPG-2 Myreth-10 Adipate, Aloe Barbadensis Leaf Juice, PEG-120 Methyl Glucose Dioleate, Sucrose, Sodium Hyaluronate, Cetyl Triethylmonium Dimethicone PEG-8 Succinate, Butylene Glycol, Hexylene Glycol, Polyquaternium-7, Laureth-2, Caprylyl Glycol, Sodium Sulfate, Tocopheryl Acetate, EDTA, Disodium EDTA, Phenoxyethanol.Please be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients.]",23.0,601,18.50
19,Philosophy,microdelivery-exfoliating-facial-wash,/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,xlsImpprod15641100,https://www.ulta.com/microdelivery-exfoliating-facial-wash?productId=xlsImpprod15641100,672 reviews,4.00,Price$28.00,"[Aqua/Water/Eau, Solum Diatomeae/Diatomaceous Earth/Terre De Diatomees, Sodium Cocoyl Apple Amino Acids, Glycerin,Acrylates/Steareth-20 Methacrylate Copolymer, Acrylates Copolymer, Decyl Glucoside, Disodium Cocoamphodiacetate, Phenoxyethanol, Lauryl Glucoside, Lithium Magnesium Sodium Silicate, Polysorbate 20, Hexylene Glycol, Sodium Hydroxide, Citrus Aurantium Dulcis (Orange) Peel Extract, Limonene, Decylene Glycol, Ethylhexylglycerin, Panthenol, Tetrasodium Edta, 1,2-Hexanediol, Asco

In [ ]:
for element in final_product_df['ingredients']:
    for k in element:
        final_product_df.append(merged_product_pd[k])